# Handling multiple sequences
- https://huggingface.co/learn/nlp-course/chapter2/5?fw=pt

이전까지, 가장 간단한 사용법을 보았다. 하나의 짧은 sequence에 대해서 inference를 하였는데, 다음과 같은 의문점들이 있다.
- 어떻게 multiple sequence를 다루는가?
- 어떻게 다른 길이의 multiple sequence를 다루는가?
- vocabulary indices는 오직 model이 잘 다루도록 하는 input일 뿐인가? (이 부분은 잘 이해를 못했다.)
- 너무 긴 seqeunce라는 것도 있는가?

어떠한 종류의 문제들이 있고 어떻게 transformer API를 사용해서 해결할지 보자.

## 1. Models expect a batch of inputs

이전 section에서, 우리는 어떻게 sequences가 숫자의 list로 변환되는지 보았다. 이번에는 숫자 list를 tensor로 변환해서 model로 보내보자.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)

model(input_ids)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

IndexError: too many indices for tensor of dimension 1

- 에러가 나왔다. 무슨일일까?

- 문제는 single sequence를 모델에 보냈다는 것이다.  
 Transformer 라이브러리의 모델들은 default로 multiple sentences를 보내는 것을 expect한다.  
 여기서는 tokenizer가 sequence에 적용되어 뒤에서 수행하는 작업을 직접 해보았는데, 자세히 보면 tensor로 변환한 것 뿐만 아니라 그 위에 dimension을 추가했음을 알 수 있다.



In [ ]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


이번엔 새로운 dimension을 추가해서 다시 시도해보자.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids.shape, input_ids)

output = model(input_ids)
print("Logits:", output.logits)


Input IDs: torch.Size([1, 14]) tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


- Batching은 multiple sequence를 model로 한 번에 보내는 것이다.  
만약 1개의 sequence밖에 없다면, batch를 single sequence로 만들면 된다.

In [ ]:
# Try it out!
batched_ids = [ids, ids]
batched_inputs = torch.tensor(batched_ids)
print(batched_inputs)

model(batched_inputs)

tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012],
        [ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])


SequenceClassifierOutput(loss=None, logits=tensor([[-2.7276,  2.8789],
        [-2.7276,  2.8789]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Batching은 모델이 multiple sequence를 다룰 수 있게 해준다.
Multiple sequence를 사용하는 것은 batch를 만드는 것 만큼이나 간단하다.


**하지만 또다른 issue가 있다.**   

바로 여러 개의 sequence마다 길이가 다른 경우이다.  
만약 tensor로 작업을 하면, 우리는 반드시 rectangular shape로 만들어야 한다. 이를 해결하기 위해 `pad`라는 것을 추가할 것이다.

## 2. Padding the inputs

다음과 같은 list of lists는 tensor로 변환될 수 없다. 왜냐하면 둘의 길이가 다르기 때문이다.

In [ ]:
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

torch.tensor(batched_ids)

ValueError: expected sequence of length 3 at dim 1 (got 2)

이 문제를 다루기 위해, 우리는 `padding`을 사용해서 tensor를 rectangular하게 만들 것이다.  
padding은 sentence가 같은 길이를 가지도록 한다.  
이 때, padding token은 길이가 작은 sequence에 추가해서 문장의 길이가 같아지도록 한다.  
만약 10개의 문장은 단어 10개를, 문장 1개는 20개의 단어를 가진다고 하면, padding은 모든 문장을 20개의 단어를 가지게 할 것이다.

예를 들어 위의 예시에 padding을 추가한다면

In [ ]:
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

torch.tensor(batched_ids)

tensor([[200, 200, 200],
        [200, 200, 100]])

- 와 같이 된다.

Huggingface에서는 `tokenizer.pad_toekn_id`를 통해서 padding token ID를 찾을 수 있다. 이를 두 문장에 대해서 적용해보고 model에 전달해보자.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


위의 결과에서 이상한 점이 하나 있다. sequence1_ids와 sequence2_ids는 단일 sequence로 넣은 것이고, batched_ids는 sequence1_ids와 sequence2_ids를 동시에 넣은 것 뿐이지, 둘은 같은 결과를 내야한다.

batched_ids의 첫번째 줄의 결과는 sequence1_ids와 같아야하고 batched_ids의 두번째 줄의 결과는 sequence2_ids와 같아야 한다.

그런데, batched_ids의 두번째 줄과 sequence2_ids는 결과가 다르다. 그 이유는 transformer 모델에 있는 attention layer 때문이다. 이는 padding token까지 고려하기 때문인데, sequence2_ids와 batched_ids가 같은 결과를 내려면, attention layer가 padding token을 무시하도록 만들 필요가 있다. 이는 `attention mask` 라는 것을 적용해서 해결할 수 있다.


## 3. Attention masks

`Attention masks`는 input IDs tensor와 동일한 shape를 가진 tensor이고, 안의 값은 오직 0아니면 1로 이루어져있다. 1은 해당 위치에 있는 token이 attention layer에서 attend되어야 하는 부분이라는 것을 가르키고, 0은 attend해선 안되는 부분이라는 것을 가르킨다.

이전 예시에 attention mask를 적용해서 완성해본다.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]


outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(outputs)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
SequenceClassifierOutput(loss=None, logits=tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


- 이제 동일한 결과가 나왔다.



## 4. 더 긴 sequence에 대해서

Transformer 모델의 경우, sequence length의 길이에 대해 한계점이 있다. 대부분의 모델은 최대 512 혹은 1024개의 토큰까지 다룰 수 있고, 만약 더 긴 sequence를 요구하면 crash가 일어날 수 있다. 이에 대해 2가지 solution이 있다.

1. 더 긴 sequence 길이를 지원하는 모델을 사용한다.
2. 사용자의 sequence를 중간까지만 받고 자른다.

모델마다 서로 다른 sequence 길이를 받을 수 있고 몇몇 긴 sequence에 특화된 모델도 있다. Longformer와 LED가 긴 문장을 다룰 수 있는 모델의 예시이다. 만약 매우 긴 sequence를 요구하는 task를 다룬다면, 이러한 모델들을 보는 것도 좋다.

그렇지 않으면 문장을 중간에 자르는 것인데, 이러한 경우에는 `max_sequence_length`를 사용할 수 있다.

In [ ]:
sequence = sequence[:max_sequence_length]

## 튜토리얼 영상

1. Truncate는 중요한 정보의 손실을 가져올 수 있다.  
2. padding 토큰은 아무거나 쓰는 것이 아닌, pre training을 할 때 사용한 것을 사용해야한다.
3. 우리가 굳이 padding token 넣지 않아도 `AutoTokenizer`를 사용할 때 `tokenizer(sentences, padding=True)`를 하면 padding을 포함한 tokenizing이 된다.